
<div style="text-align:center">
    
# ESTUAIRE TECHNICAL TEST

<br></div>
**Because of my current position taking a lot of my free time during the week, I only had this weekend to work on this project, so I have a lot of rushed EDA notebooks that I don’t feel are presentable. I am using this section to summarize my findings.**

In the available data, ~64% of flights do not generate any contrails. For those that do, the CO2e impact varies greatly.  
I therefore decided to split the problem into two parts:  
- I will build a model to predict which flights will generate a contrail and which won’t.  
- Using the flights that are expected to generate contrails, a second model will estimate their kgCO2e impact.

<div style="text-align:center"><b>Understanding/researching the subject</b><br><br></div>

Using scientific papers and publicly available knowledge, I identified the factors affecting contrail creation, duration, size, and warming effect.

**Contrail Creation Causes:**

- Altitude: Needs to be high enough (typically above 8–10 km)  
- Temperature: Needs to be low enough (below approximately -40°C)  
- Humidity: Needs to be high enough (above approximately 70%)  
- Engines produce varying amounts of water vapor, which increases local humidity  
- Engines also emit particles that provide surfaces for water vapor to freeze on  
- If multiple planes have flown the same path recently and created contrails → increased local humidity  

**Contrail Size and Duration Causes:**

- Wind: Strong winds spread the contrail quickly, while little to no wind allows it to persist longer  
- Distance traveled at cruising altitude: Longer distance = longer contrail  
- Multiple planes creating contrails on the same path can merge into a larger, longer-lasting contrail (possibly forming a cirrus cloud)  
- Atmospheric stability  
- Humidity variation between atmospheric layers  

**Contrail Warming/Cooling Causes:**

- Solar radiation: Contrails reflect some sunlight back to space → cooling effect  
- Earth radiation: Contrails trap some outgoing longwave radiation → warming effect

<div style="text-align:center"><b>EDA findings / Feature engineering</b><br><br></div>


In some cases, flights can have a cooling effect on the environment. This happens when the solar radiation reflected by the contrail is higher than the upward radiation emitted by the Earth. It occurs in the morning, when the ground is at its coolest and the Sun starts sending radiation through the atmosphere.  
As the day progresses, contrails’ greenhouse effect increases and is at its worst at nightfall, when the Sun no longer emits radiation but the Earth is still warm from the day.  
In the provided dataset, on average, there are 19,329 flights with an **average 15306 kgCO2e warming effect**.  
If we cut off all flights departing after 1:00 PM, the remaining 9,283 flights have on **average a -61 kgCO2e warming effect** (which is then a cooling effect).  
The earlier the flights, the stronger the average cooling effect becomes.  

**Available data:**

- Cargo Capacity: Not a single cargo flight in our test set.
- Role: All flights are commercial flights.
- Origin Airport/Destination Airport: Useful for determining local times and latitude/longitude for precise exogenous weather/radiation data pull.
- Take-off Time (UTC): Transformed into "Take-off Time (Local)," which was useful to timestamp multiple points during the flight to pull weather data from (assuming straight-line flight between both airports and using the aircraft's average speed/climb rate).
- Date: Information already included in "Take-off Time (UTC)". It is interesting to note that summer flights have a much lower contrail impact than other seasons, probably because the nights are shorter and solar radiation is stronger, so the contrails’ cooling effect counterbalances the greenhouse effect.
- Aircraft: Used to get information about flight cruise altitude/speed/climb rate.
- Engine: Used to get particle emissions (but I could not find water vapor emissions of the engines). Although not strong, there is a clear correlation between engine types and contrail creation probability and kgCO2e.
- Registration: No correlation
- Seats: No correlation
- Flight Number: Airlines can be extracted from this. Although useless for forecasting, it could be interesting from a commercial point of view to isolate airline-specific flights. If we identify an airline that pollutes significantly more than others, we could contact them with a report outlining the probable causes and suggesting ways to reduce their environmental footprint.
- Distance OD (km): Useless since we have access to the actual distance flown, and we can already get that information from the geolocation of both airports.
- Distance Flown (km): Almost no flight under 600 km (~3,000 flights) showed any contrail, likely due to planes not reaching cruising altitude. The longer the flight, the more likely a contrail will appear (stabilizing at around 60%), while the warming effect keeps increasing as the contrails get longer.
- CO2 (kgCO2e): Weak correlation, likely due to confounding factors: more CO2 emission = longer flight = higher altitude, etc. → increased contrail likelihood but not directly due to CO2 emission

**Exogenous data:**

I decided to pull weather and radiation data from 5 points during the flight:

- When the flight first enters the minimum altitude for contrail creation
- When the flight enters its cruising altitude
- Mid-flight
- When the flight exits its cruising altitude and starts descending
- When the flight is descending past the minimum altitude for contrail creation

For those 5 coordinates and timestamps, I pulled data from ERA5 for:

- Temperature
- Humidity
- Longwave radiation (outgoing thermal energy from the Earth)
- Shortwave radiation (incoming solar energy)



<div style="text-align:center"><b>Model Building</b><br><br></div>


I chose blablabla ... fill in


In [41]:
import pandas as pd

df = pd.read_csv('final_data.csv')

print(df['is Contrail'].value_counts())
df_classification = df.drop(columns = ['Contrails (kgCO2e)'])
df_regression = df.loc[df['is Contrail']==1].drop(columns =['is Contrail'])

print(df.head())

is Contrail
False    12587
True      6742
Name: count, dtype: int64
   Distance Flown (km)  is Contrail  Cruise_Speed_kmh  Flight_Duration  \
0          1155.727519        False             820.0         1.409424   
1          1255.367953         True             820.0         1.530937   
2          1590.238353         True             820.0         1.939315   
3          1492.741633         True             830.0         1.798484   
4           278.780456        False             800.0         0.348476   

   Engine_Code  nvPM_particles_per_s  T_Threshold_1  H_Threshold_1  \
0            6             -0.158837     221.390167      40.476620   
1            6             -0.158837     224.548630      71.908676   
2            6             -0.158837     224.568924      36.682587   
3            5              1.181357     220.466171      19.305475   
4            4             -1.163983     226.943893      79.043015   

   T_Cruise_1  H_Cruise_1  T_Cruise_2  H_Cruise_2  T_Threshold_2  

In [42]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np
from sklearn.metrics import confusion_matrix
import joblib

X = df_classification.drop(columns=['is Contrail', 'Flight_Duration', 'Engine_Code', 'nvPM_particles_per_s', 'Cruise_Speed_kmh','Take-off Hour'])
y = df_classification['is Contrail']

preprocessor = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

model = make_pipeline(
    preprocessor,
    RandomForestClassifier(class_weight='balanced', random_state=0)
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)

param_dist = {
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_depth': [None, 10, 20, 30],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__min_samples_leaf': [1, 2, 4]
}

search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=3, scoring='roc_auc', n_jobs=-1, random_state=0)
search.fit(X_train, y_train)

best_model = search.best_estimator_

y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

cm = confusion_matrix(y_test, y_pred)
TP = cm[1, 1]
FP = cm[0, 1] 
TP_percentage = (TP / (TP + FP)) * 100
FP_percentage = (FP / (TP + FP)) * 100
print(f"True Positive Percentage: {TP_percentage:.2f}%")
print(f"False Positive Percentage: {FP_percentage:.2f}%")

best_rf = search.best_estimator_.named_steps['randomforestclassifier']
feature_names = X.columns
importances = best_rf.feature_importances_
top_features = sorted(zip(importances, feature_names), reverse=True)

print("\nTop Features:")
for imp, name in top_features:
    print(f"{name}: {imp:.4f}")

joblib.dump({
    'model': search.best_estimator_,
    'features': X.columns.tolist()
}, 'classifier.joblib')


              precision    recall  f1-score   support

       False       0.86      0.82      0.84      2518
        True       0.70      0.75      0.72      1348

    accuracy                           0.80      3866
   macro avg       0.78      0.79      0.78      3866
weighted avg       0.80      0.80      0.80      3866

ROC AUC Score: 0.8769
True Positive Percentage: 69.52%
False Positive Percentage: 30.48%

Top Features:
Distance Flown (km): 0.2742
H_Threshold_1: 0.1209
H_Threshold_2: 0.1005
T_Threshold_1: 0.0836
T_Threshold_2: 0.0770
H_Cruise_1: 0.0573
T_Cruise_1: 0.0544
H_Cruise_2: 0.0523
T_Cruise_2: 0.0498
W_PCA_1: 0.0479
W_PCA_2: 0.0446
LW_SW_PC: 0.0375


['classifier.joblib']

In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import joblib
X = df_regression.drop(columns = ['Contrails (kgCO2e)','Flight_Duration','Cruise_Speed_kmh','Take-off Hour'])
y = df_regression['Contrails (kgCO2e)']

preprocessor = make_pipeline(SimpleImputer(strategy='mean'), 
                             StandardScaler())

reg_model = make_pipeline(
    preprocessor,
    RandomForestRegressor(random_state=0)
)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

param_dist = {
    'randomforestregressor__n_estimators': [100, 200],
    'randomforestregressor__max_depth': [10, 20, None]
}
search = RandomizedSearchCV(reg_model, param_distributions=param_dist, n_iter=5, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
search.fit(X_train, y_train)

best_model = search.best_estimator_
y_pred = best_model.predict(X_test)

print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R²:", r2_score(y_test, y_pred))

regressor = best_model.named_steps['randomforestregressor']
feature_names = X.columns
importances = regressor.feature_importances_

for name, score in sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True):
    print(f"{name}: {score:.4f}")

    
joblib.dump({
    'model': search.best_estimator_,
    'features': X.columns.tolist()
}, 'regressor.joblib')


RMSE: 164586.77407789024
R²: 0.041224332676779385
T_Cruise_2: 0.1644
LW_SW_PC: 0.1500
Distance Flown (km): 0.1134
T_Cruise_1: 0.0904
T_Threshold_1: 0.0734
T_Threshold_2: 0.0639
nvPM_particles_per_s: 0.0564
H_Cruise_1: 0.0551
W_PCA_2: 0.0526
H_Threshold_1: 0.0526
H_Cruise_2: 0.0495
W_PCA_1: 0.0385
H_Threshold_2: 0.0348
Engine_Code: 0.0049


['regressor.joblib']